In [15]:
%load_ext autoreload

%autoreload 2

import sys
sys.path.insert(1, "..")

import pandas as pd
import numpy as np

%aimport recordlinkage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
dfA = pd.read_csv('../sampledata/dataset2.csv', index_col='rec_id', na_values=' ')
dfB = pd.read_csv('../sampledata/dataset1.csv', index_col='rec_id', na_values=' ')

print "The number of record in dataset A: %s" % len(dfA)
print "The number of record in dataset B: %s" % len(dfB)

dfB.head()

The number of record in dataset A: 5000
The number of record in dataset B: 1000


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [17]:
std_dfA = recordlinkage.StandardDataFrame(dfA)

Create an object Pairs to create candidate record pairs. The class 'Pairs' can take 1 or 2 arguments, both dataframes. If one dataframe is given, the record pairs are for deduplication. When two dataframes are given, the data is linked between two files. 

In [18]:
pairing = recordlinkage.Pairs(dfA, dfB, suffixes=('_dfA', '_dfB'))

The simplest method of comparing record pairs is to compare all possible records pairs. The method 'full' is used for this. The function takes no arguments. 

In [5]:
pairs_full = pairing.full()
print "The reduction ratio is %s" % pairing.reduction_ratio()
pairs_full.head(10)

The reduction ratio is 0.0


given_name_dfA surname_dfA  street_number_dfA  \
index_dfA    index_dfB                                                     
rec-2778-org rec-223-org            sarah       bruhn                 44   
             rec-122-org            sarah       bruhn                 44   
             rec-373-org            sarah       bruhn                 44   
             rec-10-dup-0           sarah       bruhn                 44   
             rec-227-org            sarah       bruhn                 44   
             rec-6-dup-0            sarah       bruhn                 44   
             rec-190-dup-0          sarah       bruhn                 44   
             rec-294-org            sarah       bruhn                 44   
             rec-206-dup-0          sarah       bruhn                 44   
             rec-344-org            sarah       bruhn                 44   

                            address_1_dfA address_2_dfA    suburb_dfA  \
index_dfA    index_dfB                                                  
rec-2778-org rec-223-org    forbes street    wintersloe  kellerberrin   
             rec-122-org    forbes street    wintersloe  kellerberrin   
             rec-373-org    forbes street    wintersloe  kellerberrin   
             rec-10-dup-0   forbes street    wintersloe  kellerberrin   
             rec-227-org    forbes street    wintersloe  kellerberrin   
             rec-6-dup-0    forbes street    wintersloe  kellerberrin   
             rec-190-dup-0  forbes street    wintersloe  kellerberrin   
             rec-294-org    forbes street    wintersloe  kellerberrin   
             rec-206-dup-0  forbes street    wintersloe  kellerberrin   
             rec-344-org    forbes street    wintersloe  kellerberrin   

                            postcode_dfA state_dfA  date_of_birth_dfA  \
index_dfA    index_dfB                                                  
rec-2778-org rec-223-org            4510       vic           19300213   
             rec-122-org            4510       vic           19300213   
             rec-373-org            4510       vic           19300213   
             rec-10-dup-0           4510       vic           19300213   
             rec-227-org            4510       vic           19300213   
             rec-6-dup-0            4510       vic           19300213   
             rec-190-dup-0          4510       vic           19300213   
             rec-294-org            4510       vic           19300213   
             rec-206-dup-0          4510       vic           19300213   
             rec-344-org            4510       vic           19300213   

                            soc_sec_id_dfA given_name_dfB surname_dfB  \
index_dfA    index_dfB                                                  
rec-2778-org rec-223-org           7535316            NaN      waller   
             rec-122-org           7535316        lachlan       berry   
             rec-373-org           7535316         deakin  sondergeld   
             rec-10-dup-0          7535316          kayla  harrington   
             rec-227-org           7535316           luke      purdon   
             rec-6-dup-0           7535316            NaN   trevorrow   
             rec-190-dup-0         7535316         darcie      turtur   
             rec-294-org           7535316        william      bishop   
             rec-206-dup-0         7535316            NaN    lombardi   
             rec-344-org           7535316            NaN      julius   

                            street_number_dfB      address_1_dfB  \
index_dfA    index_dfB                                             
rec-2778-org rec-223-org                    6   tullaroop street   
             rec-122-org                   69      giblin street   
             rec-373-org                   48  goldfinch circuit   
             rec-10-dup-0                 NaN     maltby circuit   
             rec-227-org                   23       ramsay place   
       

Make pairs based on a blocking key. This blocking key can be a list of blocking keys. In this case 'Surname' was used. 

In [20]:
pairs_block = pairing.block('surname')
print "The reduction ratio is %s" % pairing.reduction_ratio()

pairs_block.head(10)

The reduction ratio is 0.996435


given_name_dfA surname  street_number_dfA  \
index_dfA    index_dfB                                                 
rec-3981-org rec-104-dup-0         alicia    hope                100   
             rec-104-org           alicia    hope                100   
rec-2851-org rec-104-dup-0          shana    hope                 24   
             rec-104-org            shana    hope                 24   
rec-77-org   rec-104-dup-0           lily    hope                  2   
             rec-104-org             lily    hope                  2   
rec-3940-org rec-104-dup-0         joshua    hope                 27   
             rec-104-org           joshua    hope                 27   
rec-2905-org rec-104-dup-0            ned    hope                  6   
             rec-104-org              ned    hope                  6   

                                   address_1_dfA          address_2_dfA  \
index_dfA    index_dfB                                                    
rec-3981-org rec-104-dup-0       mansfield place                 sunset   
             rec-104-org         mansfield place                 sunset   
rec-2851-org rec-104-dup-0        burkitt street             warra view   
             rec-104-org          burkitt street             warra view   
rec-77-org   rec-104-dup-0        inkster street            druid's oak   
             rec-104-org          inkster street            druid's oak   
rec-3940-org rec-104-dup-0  southern cross drive         mindalay appts   
             rec-104-org    southern cross drive         mindalay appts   
rec-2905-org rec-104-dup-0         stokes street  carrington retirement   
             rec-104-org           stokes street  carrington retirement   

                           suburb_dfA  postcode_dfA state_dfA  \
index_dfA    index_dfB                                          
rec-3981-org rec-104-dup-0     byford          6061        sa   
             rec-104-org       byford          6061        sa   
rec-2851-org rec-104-dup-0   carnegie          3214       qld   
             rec-104-org     carnegie          3214       qld   
rec-77-org   rec-104-dup-0     berala          2320       qld   
             rec-104-org       berala          2320       qld   
rec-3940-org rec-104-dup-0    narooma          3182       nsw   
             rec-104-org      narooma          3182       nsw   
rec-2905-org rec-104-dup-0   angaston          2775        sa   
             rec-104-org     angaston          2775        sa   

                            date_of_birth_dfA  soc_sec_id_dfA given_name_dfB  \
index_dfA    index_dfB                                                         
rec-3981-org rec-104-dup-0           19421201         7934773          april   
             rec-104-org             19421201         7934773          april   
rec-2851-org rec-104-dup-0           19760709         2091042          april   
             rec-104-org             19760709         2091042          april   
rec-77-org   rec-104-dup-0           19150907         4726313          april   
             rec-104-org             19150907         4726313          april   
rec-3940-org rec-104-dup-0           19850729         1189951          april   
             rec-104-org             19850729         1189951          april   
rec-2905-org rec-104-dup-0           19660618         5806509          april   
             rec-104-org             19660618         5806509          april   

                            street_number_dfB     address_1_dfB address_2_dfB  \
index_dfA    index_dfB                                                          
rec-3981-org rec-104-dup-0                NaN  limestone avenue          west   
             rec-104-org                  148  limestone avenue          west   
rec-2851-org rec-104-dup-0                NaN  limestone avenue          west   
             rec-104-org                  148  limestone avenue          west   
rec-77-org   rec-104-dup-0       

In [ ]:
pairs_sorted = pairing.sortedneighbourhood(' surname', window=3)
print "The reduction ratio is %s" % pairing.reduction_ratio()

pairs_sorted.head(10)

In [ ]:
pairs_sorted_with_block = pairing.sortedneighbourhood(' given_name', window=3, blocking_on=[' surname'])
pairs_sorted_with_block.head(10)


In [7]:
for pairs in pairing.iterindex(recordlinkage.indexing._fullindex, 1000,1000):
    
    if not pairs.empty:
        print 'block'

block
block
block
block
block


This is the same as the following code.

In [ ]:
for pairs in pairing.iterfull(1000,1000):
    
    pass

Also blocking and sorted neighbourhood indexing can be used with iterations. The number of records pairs is not always equal for each iteration. 

In [12]:
for pairs in pairing.itersortedneighbourhood(1000,1000, 'surname', window=3):

    print 'block'

block
block
block
block
block


In [23]:
comp = recordlinkage.Compare(pairs_block)

# print pairs_block.columns
comp.exact(
         pairs_block[['given_name_dfA','address_2_dfA']], 
         pairs_block[['given_name_dfB', 'address_2_dfB']], 
         missing_value=9, 
         output='any',
         name='given_name'
)
comp.exact( 
         pairs_block['given_name_dfA'], 
         pairs_block['address_2_dfB'], 
         missing_value=9, 
         output='any',
         name='sur_name'
)
# comp.compare(comparing.exact, pairs_block[' address_2_dfA'], pairs_block[' address_2_dfB'], missing_value=np.nan , name='address')

# pairs_block.loc[pairs_block.ix[[0,1,2,3]].index,' given_name_dfA'] = 3
# print sum(pairs_block[' given_name_dfB'].isnull())

comp.comparison_vectors

given_name  sur_name
index_dfA      index_dfB                          
rec-3981-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-2851-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-77-org     rec-104-dup-0           0         0
               rec-104-org             0         0
rec-3940-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-2905-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-2275-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-2547-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-340-org    rec-104-dup-0           0         0
               rec-104-org             0         0
rec-2146-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-1834-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-673-org    rec-104-dup-0           0         0
               rec-104-org             0         0
rec-617-org    rec-104-dup-0           0         0
               rec-104-org             0         0
rec-2003-org   rec-104-dup-0           0         0
               rec-104-org             0         0
rec-63-dup-0   rec-271-org             0         0
               rec-314-dup-0           0         0
               rec-425-org             0         9
               rec-378-dup-0           0         0
...                                  ...       ...
rec-784-org    rec-233-org             0         9
               rec-233-dup-0           0         9
rec-3459-org   rec-360-org             0         0
               rec-233-org             0         9
               rec-233-dup-0           0         9
rec-3896-org   rec-360-org             0         0
               rec-233-org             0         9
               rec-233-dup-0           0         9
rec-2706-org   rec-127-org             0         0
               rec-127-dup-0           0         0
rec-599-dup-1  rec-272-org             0         0
rec-1283-org   rec-241-dup-0           0         0
               rec-241-org             0         0
rec-2039-org   rec-241-dup-0           0         0
               rec-241-org             0         0
rec-1945-dup-0 rec-322-dup-0           0         0
rec-1131-org   rec-112-dup-0           0         0
               rec-112-org             0         0
rec-3507-dup-0 rec-133-dup-0           0         0
rec-3020-org   rec-159-dup-0           0         0
               rec-159-org             0         0
rec-455-dup-1  rec-407-org             0         9
               rec-407-dup-0           0         9
rec-2941-org   rec-407-org             0         9
               rec-407-dup-0           0         9
rec-3847-org   rec-491-org             0         0
rec-1119-org   rec-473-org             0         0
rec-3990-org   rec-295-dup-0           0         0
               rec-295-org             0         0
rec-3964-dup-0 rec-85-dup-0            0         0

[17825 rows x 2 columns]

In [43]:
reload(recordlinkage)

fs = recordlinkage.FellegiSunterClassifier()

fs.estimate(
    comparison_vectors=comp.comparison_vectors, 
    start_m={'given_name':{0:0, 1:0.8, 9:0.2},'sur_name':{0:0.1, 9:0.9}},
    start_u={'given_name':{0:0.7, 1:0.1, 9: 0.2},'sur_name':{0:0.9, 9:0.1}},
    start_p=0.1,
    max_iter=20
)

print fs.est.number_of_pairs
print fs.est.m
fs.est.summary()

len(fs.auto_classify(comp.comparison_vectors))





17825
{'sur_name': {0: 2.463855551351267e-78, 9: 1.0}, 'given_name': {0: 0.0, 1: 1.7126533783729866e-07, 9: 0.99999982873466209}}


17825

In [ ]:
x = pd.Series([1,2,3,4])
y = pd.Series([1,2,3,4])

x.name= 'test'
y.name = 'red'
print x 
print y

pd.concat([x,y], axis=1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import networkx as nx

B = nx.Graph()
B.add_nodes_from([1,2], bipartite=0) # Add the node attribute "bipartite"
B.add_nodes_from([3], bipartite=1)
B.add_edges_from([(1,1), (1,2)])

nx.draw(B)

In [71]:
reload(recordlinkage)
import itertools

test_data = recordlinkage.StandardSeries(dfA[' given_name'].copy())

In [72]:
test_data.group_similar_values()

rec_id
rec-2778-org             sarah
rec-712-dup-0            jacob
rec-1321-org           brinley
rec-3004-org           aleisha
rec-1384-org             ethan
rec-3981-org            alicia
rec-916-org           benjamin
rec-1684-org          petreece
rec-63-dup-0            olivia
rec-3808-org               NaN
rec-112-org             joshua
rec-3297-org           rachael
rec-1315-org            joseph
rec-1050-org             sarah
rec-2116-org           sidonie
rec-3232-org            andrew
rec-1900-dup-1           kiara
rec-2460-dup-2        nicholas
rec-3123-org          isabella
rec-2166-org         alexandra
rec-1155-org           matthew
rec-1485-org           michael
rec-2852-org           georgia
rec-707-org                tia
rec-3499-org            brooke
rec-2153-org              liam
rec-1665-dup-2          jamesr
rec-35-dup-1             darcy
rec-78-org                kody
rec-3116-org             james
                      ...     
rec-3650-org            samuel
r

In [59]:
%timeit list(itertools.combinations(test_data[test_data.notnull()].astype(unicode).unique(), 2))

The slowest run took 4.19 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 255 µs per loop
